In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [16]:
model=load_model('model.h5')
with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender=pickle.load(f)
with open('one_hot_encoder_geo.pkl','rb') as f:
    one_hot_encoder_geo=pickle.load(f)
with open('scaler.pkl','rb') as f:
    scaler=pickle.load(f)

In [17]:
# Example input data
input_data = {
    'CreditScore': 0,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 56,
    'Tenure': 0,
    'Balance': 1000,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 0,
    'EstimatedSalary': 0
}

In [18]:
df=pd.DataFrame([input_data])

In [19]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,France,Male,56,0,1000,1,0,0,0


In [20]:
df['Gender']=label_encoder_gender.transform(df['Gender'])

In [21]:
geo=one_hot_encoder_geo.transform(df[['Geography']]).toarray()
geo


array([[1., 0., 0.]])

In [22]:
geo=pd.DataFrame(geo,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [23]:
geo

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [24]:
df.drop('Geography',axis=1,inplace=True)
df=pd.concat([df,geo],axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0,1,56,0,1000,1,0,0,0,1.0,0.0,0.0


In [25]:
df=scaler.transform(df)

In [26]:
df

array([[-6.70646903,  0.91601335,  1.64088425, -1.72346566, -1.19973729,
        -0.92159124, -1.55619021, -1.03227043, -1.74107203,  0.98560362,
        -0.5698444 , -0.57369368]])

In [27]:
model.predict(df)

1/1 [==============================] - 0s 83ms/step


array([[0.77724415]], dtype=float32)

In [28]:
probability=model.predict(df)[0][0]

if probability<0.5:
    print("The customer is not likely to leave the bank")
else:
    print("The customer is likely to leave the bank")

1/1 [==============================] - 0s 25ms/step
The customer is likely to leave the bank
